In [70]:
# Import required modules
# To install pandas type:
# python3.7 -m pip install pandas
# In case you do not have pip installed(Google and install it!)
import pandas
import os
import numpy

In [156]:
# Read Input Files
# file1: Excel sheet with Diseases as Column Headers containing relevant SNPs
# file2: Excel sheet with CellLines as Column Headers and Rows as SNPs

# Defining a Function for taking input
def user_input():
    # Check for correct path and existence of Directory
    pwd = input("Enter the path to work directory: ")
    assert (os.path.exists(pwd)), "This path does not exist." 
    assert (os.path.isdir(pwd)), "This is not a directory."
    os.chdir(pwd)
    file1 = input("Enter the excel file with Conditions and SNPs: ")
    assert (os.path.isfile(file1)), "This file does not exist."
    file2 = input("Enter the excel file with SNP Array Data for each Cell Line: ")
    assert (os.path.isfile(file2)), "This file does not exist."
    return (pwd, file1, file2)

In [157]:
#Defining a function to take file1 and file2 to get excel file containing each cell line for each disease for each SNP
def files_process(input_list):
    pwd = input_list[0]
    file1 = input_list[1]
    file2 = input_list[2]
    dfa = pandas.read_excel(file1, header = 0)
    print("File1 has been read.")
    dfb = pandas.read_excel(file2, header = 0)
    print("File2 has been read.")
    dfb = dfb.set_index("Sample_ID")
    
    # Create a list with all the snps from all the conditions
    conditions_list = dfa.columns
    snp_list = []
    for i in conditions_list:
        snp_list = snp_list + (dfa[i].tolist())
    # Remove NaN from the list and sort it
    snp_list = [snp_list for snp_list in snp_list if str(snp_list) != 'nan']
    snp_list1 = sorted(list(set(snp_list)))
    # Create a dataframe with all the snps from all the conditions
    dfb1 = dfb[dfb.index.isin(snp_list1)]

    # Create empty dataframe for populating snps for each condition and each cell line 
    dfa_colnames = list(dfa.columns)
    df1 = pandas.DataFrame()
    df1["Sample_ID"] = dfb1.index
    df1 = df1.set_index("Sample_ID")
    for cellLine in dfb.columns:
        df_cell = dfb[cellLine]
        for condition in dfa_colnames:
            cond_list = list(dfa[condition])
            cond_list = [cond_list for cond_list in cond_list if str(cond_list) != 'nan']
            df_cell_cond = df_cell[df_cell.index.isin(cond_list)]
            colname = str(str(cellLine) + "_" + str(condition))
            df1[colname] = df_cell_cond
    
    # Create a list for Multi-level columns
    cellLines = list(dfb1.columns)
    conditions = list(dfa.columns)
    list1 = []
    for i in range(0, len(cellLines)):
        for a in range(0,3):
            list1.append(i) 
    list2 = []
    for i in range(0, len(cellLines)):
        for a in [0,1,2]:
            list2.append(a) 

    # Rename the columns for Multiple levels        
    colnames = pandas.MultiIndex(levels = [cellLines, conditions], codes = [list1, list2])
    # Remove NaN and "AA" from the dataframe
    df2 = df1.replace(to_replace = "AA", value = "")
    df3 = df2.replace(to_replace = numpy.nan, value = "", regex = True)
    df3.columns = colnames
    return df3

In [159]:
# define a function to take a pandas dataframe and write it to an excel file
def write_to_excel(input_df):
    df1 = input_df
    output_file = input("Name of the Output File:")
    output_filename = str(output_file) + ".xlsx"
    df1.to_excel(output_filename, header = True, index = True)
    print("The output file is ready.")

In [160]:
# Call the functions
write_to_excel(files_process(user_input()))

Enter the path to work directory: /Users/sha6hg/Desktop/HasanSNPanalysis
Enter the excel file with Conditions and SNPs: /Users/sha6hg/Desktop/HasanSNPanalysis/SNPs from dB.xlsx
Enter the excel file with SNP Array Data for each Cell Line: /Users/sha6hg/Desktop/HasanSNPanalysis/#001 GenomeStudioReport_DNACore_withTkDa-1_v1.4_and_withCCHMC003_v1.xlsx
File1 has been read.
File2 has been read.
The output file is ready.


In [ ]:
# Trial files
/Users/sha6hg/Desktop/HasanSNPanalysis
/Users/sha6hg/Desktop/HasanSNPanalysis/SNPs from dB.xlsx
/Users/sha6hg/Desktop/HasanSNPanalysis/#001 GenomeStudioReport_DNACore_withTkDa-1_v1.4_and_withCCHMC003_v1.xlsx